# Linear Optimization (CS5040) : Assignment 04

# Team:

#### 1. CS23MTECH11004 &emsp; - &emsp; Ashish B Emmanuel
#### 2. CS23MTECH11016 &emsp; - &emsp; Raghavendra Kulkarni
#### 3. CS23MTECH14013 &emsp; - &emsp; Shagun
#### 4. CS23MTECH14016 &emsp; - &emsp; Trishita Saha

# Question:

Implement the simplex algorithm to maximize the objective function under given constraints using Vertex Marching method and assuming:
* Rank of A is n

# Input Format:

Input is a .csv file with m+1 rows and n+1 columns. 
* The first row excluding the last element is the cost vector c of length n
* The last column excluding the top element is the constraint vector b of length m
* Rows two to m+1 and column one to n is the matrix A of size m*n


# Solution:

## Import the required libraries

In [1]:
import numpy as np
from sympy import Matrix, zoo

## Function to read the .csv Input File

In [2]:
'''
Name        :   readCSV()
Description :   Reads the input matrix A, vectors b and c from a CSV file.
Argument    :   Name of the Input file (without path)
Return      :   The matrix A, vectors b and c in that order
'''

def readCSV(filename):
    
    # Open the file and assign a File Handler
    with open(filename) as input:

        # Read the first line to extract vector c
        line = input.readline()
        c = list(map(float, line.split(',')))
        c = np.asarray(c[:-1], dtype = float)

        # Read all the remaining lines to extract matrix A and vector b
        A, b = [], []
        for line in input.readlines():
            l = list(map(float, line.split(',')))
            A.append(l[:-1])
            b.append(l[-1])
        A = np.asarray(A, dtype = float)
        b = np.asarray(b, dtype = float)
    
    # Return the matrix and the vectors
    return A, b, c

## Function to test the feasibility of a given point

In [3]:
'''
Name        :   testFeasibility()
Description :   Checks whether a point z is a feasible solution for the system or not.
Argument    :   Matrix A, vector b and the point z to be checked
Return      :   True, if z is feasible
                False, otherwise
'''

def testFeasibility(A, b, z):
    
    # Multiply A with z and compare it with b. Return the truth value
    return all(np.matmul(A, z) <= b)

## Function to split the matrix into tight and untight rows

In [4]:
'''
Name        :   splitRows()
Description :   Splits the matrices A and b into set of tight and untight rows with respect to a given point z.
Argument    :   Matrix A, vector b and the point z
Return      :   Returns the set of tight and untight rows of A and b in that order
'''

def splitRows(A, b, z):
    
    # Initialize required variables
    A1, A2, b1, b2 = [], [], [], []

    # Multiply matrix A with z
    Az = np.dot(A,z)

    # For every element in product Az
    for i in range(len(Az)):

        # Compare with corresponding element of vector b and decide tight or untight
        if np.round(Az[i], 3) == np.round(b[i], 3):
            A1.append(A[i])
            b1.append(b[i])
        else:
            A2.append(A[i])
            b2.append(b[i])
    
    # Return the tight and untight rows 
    return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

## Function to test whether the given point is a vertex or not

In [5]:
'''
Name        :   testVertex()
Description :   Checks whether a point z is a vertex of the feasible region or not.
Argument    :   Matrix A, vector b and the point z to be checked
Return      :   True, if z is vertex
                False, otherwise
'''

def testVertex(A, b, z):

    # Split the matrix into tight and untight rows
    A1, _, _, _ = splitRows(A, b, z)

    # Compute the rank of matrix A and compare it with number of tight rows
    return np.linalg.matrix_rank(A) <= A1.shape[0]

## Function to update a feasible point and move it towards a vertex

In [6]:
'''
Name        :   move()
Description :   Moves the feasible point towards a vertex of the feasible region by one step size.
Argument    :   Matrix A, vector b and the vertex z
Return      :   The point z reached after one step move
'''

def move(A, b, z):

    # Determine the shape of the matrix and split it into tight and untight rows
    _, n = A.shape[0], A.shape[1]
    A1, _, A2, b2 = splitRows(A, b, z)
    A1 = A1.reshape((A1.shape[0], n))

    # Compute the direction to move the feasible point
    nullspace = np.array(Matrix(A1).nullspace())
    scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
    vector = np.array(np.sum(nullspace * scale, axis = 0)).astype(float)
    u = vector / np.linalg.norm(vector)

    # Compute the step size 'alpha'
    temp = np.dot(A2 , u)
    for i in range(len(temp)):
        if temp[i] == 0:
            temp[i] = pow(2, -16)
    alpha = (b2 -np.dot(A2, z))/temp.T
    alpha = alpha[0]
    alpha = np.array([a for a in alpha if not ((a > 10**6) or (-1*a) > 10**6)])
    alpha = alpha[np.argmin(np.abs(alpha.copy()))]

    # Update the feasible point by one step size
    u = u.T
    z = z + alpha * u
    z = z[0]

    # Return the updated feasible point
    return z

## Function to test whether the given vertex is degenerate

In [7]:
'''
Name        :   testDegeneracy()
Description :   Checks whether a vertex z of the feasible region is degenerate.
Argument    :   Matrix A, vector b and the point z to be checked
Return      :   True, if vertex z is degenerate
                False, otherwise
'''

def testDegeneracy(A, b, z):

    # Split the matrix into tight and untight rows
    A1, _, _, _ = splitRows(A, b, z)

    # Compute the rank of matrix A and compare it with number of tight rows
    return np.linalg.matrix_rank(A) < A1.shape[0]

## Function to remove degeneracy from the feasible region

In [8]:
'''
Name        :   removeDegeneracy()
Description :   Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
Argument    :   The vector b
Return      :   The modified vector b
'''

def removeDegeneracy(b, epsilon):

    for i in range(len(b)):
        b[i] += pow(epsilon, i + 1)

    # Return the updated vector b 
    return b

## Function to test whether the given vertex is an optimum vertex or not

In [9]:
'''
Name        :   testOptimum()
Description :   Checks whether a vertex z is an optimum vertex or not.
Argument    :   Matrix A, vector b and the vertex z to be checked
Return      :   True, if z is optimum vertex
                False, otherwise
'''

def testOptimum(A, b, z, c):
    
    # Split the matrix A into tight and untight rows
    A1, _, _, _ = splitRows(A, b, z)

    # Express the cost vector as a linear combination of tight rows
    beta = np.dot(np.linalg.inv(A1.T), c.T)

    # Check for non-negativity of all the coefficients and return the result
    return all([x >= 0 for x in beta])

## Function to march from one vertex to other optimizing the cost function

In [10]:
'''
Name        :   optimize()
Description :   Optimizes the cost function by moving to the next optimum vertex
Argument    :   Matrix A, vector b, the vertex z and the cost vector c
Return      :   The next optimum vertex
'''

def optimize(A, b, z, c):

    # Split the matrix into tight and untight rows
    A1, _, A2, b2 = splitRows(A, b, z)

    # Compute the direction to move in order to reach a more optimum vertex
    u = None
    A1_inverse = np.linalg.inv(A1)
    A1_inverse = np.transpose(A1_inverse)
    for x in A1_inverse:
        if np.dot((-1)*x, c.T) > 0:
            u = x.T
            break
    
    # Compute the step size 'alpha'
    temp = np.dot(A2, u)
    for i in range(len(temp)):
        if temp[i] == 0:
            temp[i] = 1e-16
    alpha = (np.dot(A2, z) - b2)/temp
    alpha = [x for x in alpha if x != zoo]
    if all([a < 0 for a in alpha]):
        return z, False
    alpha = min([x for x in alpha if x >= 0])
    
    # Update the vertex
    z = z - alpha * u

    # Return the updated vertex
    return z, True

## Function to find a feasible point of the given region

In [11]:
'''
Name        :   findFeasiblePoint()
Description :   Finds a point z that is inside the feasible region of the given system.
Argument    :   Matrix A and the vector b
Return      :   The feasible point z, if one exists
                Empty List, otherwise
'''

def findFeasiblePoint(A, b):
    m, n = A.shape
    
    # Check if the origin is a feasible point
    if all([x >= 0 for x in b]):
        return np.zeros(n, dtype = float)
    
    # Formulate a new optimization problem
    P = np.empty((m + 1, n + 1), dtype = float)    
    for x in A:
        x = np.append(x, 1)
        P = np.vstack((P, x))
    zeros = np.zeros(n + 1)
    zeros[n] = -1
    P = np.vstack((P, zeros))        
    P = P[m + 1:]
    temp = b
    q = np.append(temp, -1*np.min(b))
    z = np.zeros(n, dtype = float)
    z = np.append(z, np.min(b))
    r = np.zeros(n, dtype = float)
    r = np.append(r, 1)

    # Step 1a: Check for feasibility of the Initial Point
    if not testFeasibility(P, q, z):
        return []

    solved = False
    epsilon = pow(2, -5)
    q_org = q

    while not solved:

        # Step 1b: Move towards the first vertex
        while not testVertex(P, q, z):
            z = move(P, q, z)

        # Test for degeneracy of the vertex
        if testDegeneracy(P, q, z):
            q = removeDegeneracy(q_org.copy(), epsilon)
            epsilon *= 2
            continue

        # Step 1c: March through every vertex and check for optimum along with degeneracy
        solution = True
        degenerate = False
        while solution and (not testOptimum(P, q, z, r)):
            
            z, solution = optimize(P, q, z, r)

            # If the new polytope is unbounded reverse the direction and find the feasible point
            if not solution:
                P1, _, P2, q2 = splitRows(P, q, z)
                u = None
                P1_inverse = np.linalg.inv(P1)
                P1_inverse = np.transpose(P1_inverse)
                for x in P1_inverse:
                    if np.dot((-1)*x, r.T) > 0:
                        u = x.T
                        break
                temp = np.dot(P2, u)
                for i in range(len(temp)):
                    if temp[i] == 0:
                        temp[i] = 1e-16
                alpha = (q2 - np.dot(P2, z))/temp
                alpha = min([a for a in alpha if a >= 0])
                z = z - alpha * u

                # Check for feasiblity of the point
                if z[-1] < 0:
                    z = []
                else:
                    z = z[:-1]
                break
            
            elif testDegeneracy(P, q, z):
                q = removeDegeneracy(q_org.copy(), epsilon)
                degenerate = True
                epsilon *= 2
                break
            
            else:
                continue

        if not degenerate:
            solved = True

    # Convert the solution back to the original solution, if it exists
    if solution:
        X, y = [], []
        for i in range(len(P)):
            if np.round(np.dot(P[i], z), 3) == np.round(q[i], 3):
                X.append(P[i])
                y.append(q_org[i])
        z = np.dot(np.linalg.inv(X), y)

        # Check for feasiblity of the point
        if z[-1] < 0:
            z = []
        else:
            z = z[:-1]
    
    return z

## The driver main function executing the Simplex Algorithm steps sequentially

In [12]:
'''
Name        :   main()
Description :   Driver function which calls the subroutines according to the Simplex Algorithm.
Argument    :   None
Return      :   None
'''
def main():

    ## Read the input from the CSV file
    A, b, c = readCSV('Assignment4.csv')

    # Step 1: Find an initial feasible point
    z = findFeasiblePoint(A, b)
    if len(z) == 0:
        print('No Solution')
        return
    
    # Initial feasible point found
    solved = False
    epsilon = pow(2, -5)
    b_org = b.copy()

    while not solved:

        # Step 2: Move towards the first vertex
        while (not testVertex(A, b, z)):
            z = move(A, b, z)

        # Test for degeneracy of the vertex
        if testDegeneracy(A, b, z):
            b = removeDegeneracy(b_org.copy(), epsilon)
            epsilon *= 2
            continue

        # Print the value of the Cost function at this first vertex
        print('Vertex {} --> Cost: {}'.format(z, np.dot(z, c)))

        # Step 3: March through every vertex and check for optimum along with degeneracy
        solution = True
        degenerate = False
        while solution and (not testOptimum(A, b, z, c)):
            z, solution = optimize(A, b, z, c)
            if not solution:
                print('No Solution')
            elif testDegeneracy(A, b, z):
                b = removeDegeneracy(b_org.copy(), epsilon)
                degenerate = True
                epsilon *= 2
                break
            else:
                print('Vertex {} --> Cost: {}'.format(z, np.dot(z, c)))

        if not degenerate:
            solved = True

    # Convert the solution back to the original solution, if it exists
    if solution:
        X, y = [], []
        for i in range(len(A)):
            if np.round(np.dot(A[i], z), 3) == np.round(b[i], 3):
                X.append(A[i])
                y.append(b_org[i])
        z = np.dot(np.linalg.inv(X), y)
        print('Original Solution: Vertex {} --> Cost: {}'.format(z, np.dot(z, c)))
    
    # Return
    return

## Invoking the driver main function

In [13]:
main()

Vertex [1. 3.] --> Cost: -15.0
Vertex [3. 1.] --> Cost: -13.0
Vertex [2.96875 1.     ] --> Cost: -12.90625
Original Solution: Vertex [3. 1.] --> Cost: -13.0
